In [41]:
#some imports
from bs4 import BeautifulSoup
from dateutil import parser
from difflib import SequenceMatcher
import re,os
import glob


In [42]:
#FOR TOPIC MODELLING
#https://de.dariah.eu/tatom/topic_model_python.html
import numpy as np  # a conventional alias

import sklearn.feature_extraction.text as text
from sklearn import decomposition

In [43]:
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

In [44]:
def language_detection_with_pyenchant(string_to_read):
    #https://stackoverflow.com/questions/3788870/how-to-check-if-a-word-is-an-english-word-with-python
    import enchant
    lg_ang=0
    us = enchant.Dict("en_US")
    #print "US LOADED"
    fr = enchant.Dict("fr_FR")
    #print "FR LOADED"
    lg_fr=0
    lg_ang=0
    #print "string_to_read",string_to_read
    for word in string_to_read.split():
        #print fr.check(word)
        #print word
        if fr.check(word) == True:
            lg_fr+=1
        if us.check(word) == True:
            lg_ang+=1
    #print "THERE I AM"
    if lg_fr >= lg_ang :
        return "french"
    else:
        if lg_ang > lg_fr :
        
            return "english"
        else: 
            return "NEITHER ENGLISH NOR FRENCH"
    
    

In [45]:
def topic_analysis_with_mallet(string_to_read):
    lng = language_detection_with_pyenchant(string_to_read)
    if lng== "french" :
        lng= ["a","à","â","abord","afin","ah","ai","aie","ainsi","allaient","allo","allô","allons","après","assez","attendu","au","aucun","aucune","aujourd","aujourd'hui","auquel","aura","auront","aussi","autre","autres","aux","auxquelles","auxquels","avaient","avais","avait","avant","avec","avoir","ayant","b","bah","beaucoup","bien","bigre","boum","bravo","brrr","c","ça","car","ce","ceci","cela","celle","celle-ci","celle-là","celles","celles-ci","celles-là","celui","celui-ci","celui-là","cent","cependant","certain","certaine","certaines","certains","certes","ces","cet","cette","ceux","ceux-ci","ceuxlà","chacun","chaque","cher","chère","chères","chers","chez","chiche","chut","ci","cinq","cinquantaine","cinquante","cinquantième","cinquième","clac","clic","combien","comme","comment","compris","concernant","contre","couic","crac","d","da","dans","de","debout","dedans","dehors","delà","depuis","derrière","des","dès","désormais","desquelles","desquels","dessous","dessus","deux","deuxième","deuxièmement","devant","devers","devra","différent","différente","différentes","différents","dire","divers","diverse","diverses","dix","dix-huit","dixième","dix-neuf","dixsept","doit","doivent","donc","dont","douze","douzième","dring","du","duquel","durant","e","effet","eh","elle","elle-même","elles","ellesmêmes","en","encore","entre","envers","environ","es","ès","est","et","etant","étaient","étais","était","étant","etc","été","etre","être","eu","euh","eux","eux-mêmes","excepté","f","façon","fais","faisaient","faisant","fait","feront","fi","flac","floc","font","g","gens","h","ha","hé","hein","hélas","hem","hep","hi","ho","holà","hop","hormis","hors","hou","houp","hue","hui","huit","huitième","hum","hurrah","i","il","ils","importe","j","je","jusqu","jusque","k","l","la","là","laquelle","las","le","lequel","les","lès","lesquelles","lesquels","leur","leurs","longtemps","lorsque","lui","lui-même","m","ma","maint","mais","malgré","me","même","mêmes","merci","mes","mien","mienne","miennes","miens","mille","mince","moi","moi-même","moins","mon","moyennant","n","na","ne","néanmoins","neuf","neuvième","ni","nombreuses","nombreux","non","nos","notre","nôtre","nôtres","nous","nous-mêmes","nul","o","o|","ô","oh","ohé","olé","ollé","on","ont","onze","onzième","ore","ou","où","ouf","ouias","oust","ouste","outre","p","paf","pan","par","parmi","partant","particulier","particulière","particulièrement","pas","passé","pendant","personne","peu","peut","peuvent","peux","pff","pfft","pfut","pif","plein","plouf","plus","plusieurs","plutôt","pouah","pour","pourquoi","premier","première","premièrement","près","proche","psitt","puisque","q","qu","quand","quant","quanta","quant-à-soi","quarante","quatorze","quatre","quatre- vingt","quatrième","quatrièmement","que","quel","quelconque","quelle","quelles","quelque","quelques","quelqu'un","quels","qui","quiconque","quinze","quoi","quoique","r","revoici","revoilà","rien","s","sa","sacrebleu","sans","sapristi","sauf","se","seize","selon","sept","septième","sera","seront","ses","si","sien","sienne","siennes","siens","sinon","six","sixième","soi","soi-même","soit","soixante","son","sont","sous","stop","suis","suivant","sur","surtout","t","ta","tac","tant","te","té","tel","telle","tellement","telles","tels","tenant","tes","tic","tien","tienne","tiennes","tiens","toc","toi","toi-même","ton","touchant","toujours","tous","tout","toute","toutes","treize","trente","très","trois","troisième","troisièmement","trop","tsoin","tsouin","tu","u","un","une","unes","uns","v","va","vais","vas","vé","vers","via","vif","vifs","vingt","vivat","vive","vives","vlan","voici","voilà","vont","vos","votre","vôtre","vôtres","vous","vous-mêmes","vu","w","x","y","z","zut"]
    
    vectorizer = text.CountVectorizer(input=string_to_read, stop_words=lng, min_df=3)
    dtm = vectorizer.fit_transform(string_to_read.split()).toarray()
    vocab = np.array(vectorizer.get_feature_names())
    dtm.shape
    len(vocab)
    num_topics = 20
    num_top_words = 20
    clf = decomposition.NMF(n_components=num_topics, random_state=1)
    doctopic = clf.fit_transform(dtm)
    topic_words = []
    for topic in clf.components_:
        word_idx = np.argsort(topic)[::-1][0:num_top_words]
        topic_words.append([vocab[i] for i in word_idx])
    doctopic = doctopic / np.sum(doctopic, axis=1, keepdims=True)
    return [topic_words,doctopic]

In [46]:
#so far french support
def find_geo_topics(string_to_read, lng):
    geo=0
    geo_words=[]
    #print string_to_read
    if lng== "french":
        for word in (string_to_read.split()):
        #print word
        #LUSSAULT ;) https://www.espacestemps.net/articles/entrees-par-categories/
            # if (word in ["Théorie de l’espace","Accessibilité","Acteur spatial","Action spatiale","Agencement","Agglomération","Ailleurs","Aire","Aire culturelle","Alignement","Ambiance architecturale et urbaine","Aménagement du territoire","Anamorphose","Anthropisation","Archipel mégalopolitain mondial ","Armature urbaine","Attraction","Autocorrélation spatiale","Banlieue","Campagne","Capital spatial","Carte","Carte mentale","Centralité","Centre/Périphérie","Centre urbain","Chorème","Chorotype","Circulation","Citadinité","Communication territoriale","Commutateur","Compromis territorial","Concentration","Configuration spatiale","Confins","Connexité","Contact","Contiguïté","Continent","Continuité","Coprésence","Corps","Cospatialité","Cyberespace","Décentralisation","Découpage","Découverte","Défrichement","Densité","Désert","Déterritorialisation","Développement local","Diaspora","Différenciation spatiale -Diffusion","Discontinuité","Dispositif spatial légitime","Distance","Distribution rang/taille","Distribution spatiale","District industriel","Diversité","Dynamique spatiale","Écart","Échelle","Économie-monde","Écoumène","Edge City","Emblème territorial","Emboîtement","Empire","Enclavement -Ensemble géographique","Espace","Espace public ","Espace vécu","État","État local","Étendue","Fédéralisme","Finage","Firme transnationale","Fleuve","Flux","Foncier","Forêt","Fractale","Friche","Front","Front pionnier","Frontière","Générique ","Gentrification","Géoéconomie","Géogramme","Géographicité","Géographie","Géon","Géopolitique","Géosystème","Géotype","Ghetto","Glacis","Gouvernement urbain","Gradient","Graphe","Graphique","Gravitaire ","Guerre","Habitat","Habitat non-réglementaire","Habiter","Haut lieu","Heimat","Hétérotopie","Hinterland","Horizont","Hors-sol","Hub","Identité spatiale","Île","Image","Imaginaire géographique","Immanence/Transcendance ","Infra-urbain","Interaction spatiale","Interface","Interspatialité","Irrigation","Isolat","Isotropie","Jardin","Justice spatiale","Lieu","Lieux centraux ","Limite","Littoral","Local","Localisation","Logistique","Maillage","Maison","Marchandise","Médiance","Méditerranée","Mer","Métaphore spatiale","Métrique","Métropole/Mégalopole","Métropolisation","Migration","Milieu","Milieu innovateur","Minorité territoriale","Mobilité","Monde","Mondialisation","Montagne","Nation","Network","Nœud","Norme","Oasis","Objet géographique","Parc à thème","Parc naturel","Parcours","Partie du monde","Pavillonnaire ","Pays","Paysage","Périurbain","Peuplement","Polarisation","Population ","Position","Pratique spatiale","Projet urbain","Prospective territoriale","Proxémie","Reconstruction","Reconversion","Rénovation/Restauration/Réhabilitation","Représentation de l’espace","Réseau","Réseau technique","Réseau urbain","Rhizome","Rue","Rural","Schéma d’aménagement","Ségrégation","Seuil","Site","Situation géographique","Société-Monde","Sol","Spatialité","Stratégie spatiale","Substance","Système d’Information Géographique ","Système productif local ","Système spatial","Technopôle/Technopole","Télé-communication","Télétravail","Terre","Territoire","Territorial ","Territorialité","Terroir","Topogenèse","Topographie","Topologie","Toponymie","Tourisme","Transition démographique","Transports","Ubiquité","Urbain","Urbain ","Urbanisation","Urbanité","Valeur spatiale","Vaterland","Végétation","Village","Ville","Ville mondiale","Ville nouvelle","Violence","Visibilité ","Voisinage","Zonage","Zone climatique"]) or  "géogr" in word or "géomat" in word :
            if  "géogr" in word.lower() or "géomat" in word.lower() or "spatial" in word.lower() or "urbanis" in word.lower():  #or "carto" in word:  
                geo+=1
                geo_words.append(word.lower())
    if lng== "english":
        for word in (string_to_read.split()):
        #print word
        #LUSSAULT ;) https://www.espacestemps.net/articles/entrees-par-categories/
             if  "geogr" in word.lower() or "geomat" in word.lower() or "spatial" in word.lower() or "urbanis" in word.lower(): # or "carto" in word:
                geo+=1
                geo_words.append(word.lower())
    #print "SUJETS\n",geo_words,"\n points : ",geo
    return [geo, geo_words]

    

In [47]:
def topic_analysis_with_manual_detection(string_to_read):
    lng = language_detection_with_pyenchant(string_to_read)
    if lng== "french" :
        geopoints= find_geo_topics(string_to_read, "french")
        #print string_to_read
    else:
        if lng== "english" :
            geopoints= find_geo_topics(string_to_read, "english")
        
        else:
            print "LANGUAGE NOT YET SUPPORTED"
            geopoints=['',''] 
    #    lng= ["a","à","â","abord","afin","ah","ai","aie","ainsi","allaient","allo","allô","allons","après","assez","attendu","au","aucun","aucune","aujourd","aujourd'hui","auquel","aura","auront","aussi","autre","autres","aux","auxquelles","auxquels","avaient","avais","avait","avant","avec","avoir","ayant","b","bah","beaucoup","bien","bigre","boum","bravo","brrr","c","ça","car","ce","ceci","cela","celle","celle-ci","celle-là","celles","celles-ci","celles-là","celui","celui-ci","celui-là","cent","cependant","certain","certaine","certaines","certains","certes","ces","cet","cette","ceux","ceux-ci","ceuxlà","chacun","chaque","cher","chère","chères","chers","chez","chiche","chut","ci","cinq","cinquantaine","cinquante","cinquantième","cinquième","clac","clic","combien","comme","comment","compris","concernant","contre","couic","crac","d","da","dans","de","debout","dedans","dehors","delà","depuis","derrière","des","dès","désormais","desquelles","desquels","dessous","dessus","deux","deuxième","deuxièmement","devant","devers","devra","différent","différente","différentes","différents","dire","divers","diverse","diverses","dix","dix-huit","dixième","dix-neuf","dixsept","doit","doivent","donc","dont","douze","douzième","dring","du","duquel","durant","e","effet","eh","elle","elle-même","elles","ellesmêmes","en","encore","entre","envers","environ","es","ès","est","et","etant","étaient","étais","était","étant","etc","été","etre","être","eu","euh","eux","eux-mêmes","excepté","f","façon","fais","faisaient","faisant","fait","feront","fi","flac","floc","font","g","gens","h","ha","hé","hein","hélas","hem","hep","hi","ho","holà","hop","hormis","hors","hou","houp","hue","hui","huit","huitième","hum","hurrah","i","il","ils","importe","j","je","jusqu","jusque","k","l","la","là","laquelle","las","le","lequel","les","lès","lesquelles","lesquels","leur","leurs","longtemps","lorsque","lui","lui-même","m","ma","maint","mais","malgré","me","même","mêmes","merci","mes","mien","mienne","miennes","miens","mille","mince","moi","moi-même","moins","mon","moyennant","n","na","ne","néanmoins","neuf","neuvième","ni","nombreuses","nombreux","non","nos","notre","nôtre","nôtres","nous","nous-mêmes","nul","o","o|","ô","oh","ohé","olé","ollé","on","ont","onze","onzième","ore","ou","où","ouf","ouias","oust","ouste","outre","p","paf","pan","par","parmi","partant","particulier","particulière","particulièrement","pas","passé","pendant","personne","peu","peut","peuvent","peux","pff","pfft","pfut","pif","plein","plouf","plus","plusieurs","plutôt","pouah","pour","pourquoi","premier","première","premièrement","près","proche","psitt","puisque","q","qu","quand","quant","quanta","quant-à-soi","quarante","quatorze","quatre","quatre- vingt","quatrième","quatrièmement","que","quel","quelconque","quelle","quelles","quelque","quelques","quelqu'un","quels","qui","quiconque","quinze","quoi","quoique","r","revoici","revoilà","rien","s","sa","sacrebleu","sans","sapristi","sauf","se","seize","selon","sept","septième","sera","seront","ses","si","sien","sienne","siennes","siens","sinon","six","sixième","soi","soi-même","soit","soixante","son","sont","sous","stop","suis","suivant","sur","surtout","t","ta","tac","tant","te","té","tel","telle","tellement","telles","tels","tenant","tes","tic","tien","tienne","tiennes","tiens","toc","toi","toi-même","ton","touchant","toujours","tous","tout","toute","toutes","treize","trente","très","trois","troisième","troisièmement","trop","tsoin","tsouin","tu","u","un","une","unes","uns","v","va","vais","vas","vé","vers","via","vif","vifs","vingt","vivat","vive","vives","vlan","voici","voilà","vont","vos","votre","vôtre","vôtres","vous","vous-mêmes","vu","w","x","y","z","zut"]
    
    return geopoints

In [107]:
def read_html_pages_in_dirs_and_extract_mails_dict(pages_dir):
    list_all_mails=[]
    dict_mails={}
    #rootdir="./DHFR_sample/"
    count_Cc=0
    count_message=0
    for subdir, dirs, files in os.walk(pages_dir):
#ON VA LIRE TOUS LES FICHIERS DU REP ROOTDIR
        
        for file in files:
    #V1     if file.endswith('.html') == True  and file != ("index.html") and not re.match("mail\d+.html",file) : 
           if re.match("msg\d+.html",file) :
          
            # les index.html sont des recap, on pourrait les compter pour s'assurer du nombre de messages
                
                #!print "\n",os.path.join(subdir, file), "\n"
                champs_mail={}
                filename=os.path.join(subdir, file)
                champs_mail["Geo_Topic"]=read_html_and_testtextextract(filename)
                
                
                soup=BeautifulSoup(open(filename, 'r').read(),'lxml')#, 'html.parser') 
#RECUPERER LE SUJET DU MESSAGE
                
                i=0
                for zone_cible in soup.findAll('ul'):
                    i+=1
                    sujet_messg = zone_cible.find(string=re.compile("\[DH\]"))
                    
                    zone_de_metadonnees = None
                    special_auteur=0
##ON ISOLE LE CHAMPS AUTEUR
                    if '<li><strong>From</strong>:' in str(zone_cible) and '<li><strong>To</strong>:' in str(zone_cible) :
                        #print "FROM/TO DETECTED, i= ",i
                        count_message+=1
                        #print  zone_cible
                        nom_auteur= re.sub(r'<ul>\n<li><strong>From</strong>: ', '',str(zone_cible))
                        sep = "&lt;"
                        nom_auteur = nom_auteur.split(sep, 1)[0].lower()
                        
                        
                        mail1= re.sub(r'<ul>\n<li><strong>From</strong>:.+\n.+\n.+\n document\.write\(\"', '',str(zone_cible))
                        mail2=re.sub(r'<ul>\n<li><strong>From</strong>:.+\n.+\n.+\n document\.write\(\".+@\" \+ \"', '',str(zone_cible))
                        sep1 = "\" + \"@\" + "
                        sep2 = "\")"
                        mail1= mail1.split(sep1, 1)[0]
                        mail2= mail2.split(sep2, 1)[0]
                        mail_auteur= mail1+"@"+mail2
                        mail_auteur=mail_auteur.lower()
                        #print mail_auteur
                        if len(mail_auteur)>60:
                            try:
                                mail= re.sub(r'<ul>\n<li><strong>From</strong>:', '',str(zone_cible))
                                mail=re.search('&lt;(.*)&gt;',str(mail))
                                mail=mail.group(1)
                                #print "MAILBIS",mail
                                mail_auteur=mail.lower()
                                #special_auteur=1
                            except:
                                print "BUG"
                        #CHECK PROPER CAPTURE OF THE NAME    
                        if len(nom_auteur)>50:
                            #print zone_cible
                            nom_auteur=mail_auteur
                        if len(nom_auteur)<5:
                            nom_auteur=mail_auteur
                        if nom_auteur[0]== "\"":
                            nom_auteur=nom_auteur[1:]
                            nom_auteur=nom_auteur[:-2]
                            print nom_auteur
                        if nom_auteur.endswith(' '):
                            nom_auteur=nom_auteur[:-1]
                        if  mail_auteur not in list_all_mails:
                            list_all_mails.append(mail_auteur)
                            
                        #!print "nom_auteur ",nom_auteur+" mail_auteur "+mail_auteur,"\n"
                        champs_mail["nom_auteur"]=  nom_auteur.lower()
                        champs_mail["mail_auteur"]=  mail_auteur.lower()
                        champs_mail["ref_physique_de_l_article"]=  os.path.join(subdir, file)
                        if sujet_messg is not None:
                            champs_mail["sujet_du_message"] = sujet_messg.lower()
##ON ISOLE LE CHAMP DESTINATAIRES
                        #if special_auteur == 1:
                            #print "ZCspecAut",zone_cible
                        #else: 
                        
                        destinataires=re.sub(r'<ul>\n<li><strong>From</strong>:.+\n.+\n.+\n.+\n.+\n.+\n.+\n', '',str(zone_cible))
                        
                        sep3=";</li>"
                        destinataires= destinataires.split(sep3, 1)[0]
                        #print destinataires
                        dest=re.findall("<script type=.*\n.+\n.+\n.+\n.+/script>", destinataires)
                        #print dest
                        liste_destinataires=[]
                        for destinataire in dest:
                            #print destinataire,"\n___________________\n"
                            mail1= re.sub(r'<script type=\"text/javascript\">\n <!-- \n document\.write\(\"', '',str(destinataire))
                            mail2=re.sub(r'<script type=\"text/javascript\">\n <!-- \n document\.write\(\".+@\" \+ \"', '',str(destinataire))
                            sep1 = "\" + \"@\" + "
                            sep2 = "\")"
                            mail1= mail1.split(sep1, 1)[0].lower()
                            mail2= mail2.split(sep2, 1)[0].lower()
                            mail_destinataire= mail1+"@"+mail2
                            #print mail_destinataire,"\n___________\n"
                            if mail_destinataire not in liste_destinataires :
                                liste_destinataires.append(mail_destinataire)
                            if  mail_destinataire not in list_all_mails:
                                list_all_mails.append(mail_destinataire)
                        if liste_destinataires==[]:
                            #print zone_cible
                            liste_destinataires=['dh@groupes.renater.fr']
                        #!print "destinataires: ",liste_destinataires,"\n"
                        champs_mail["liste_mails_dests"] = liste_destinataires
                       
                        if '<li><strong>Cc</strong>:' in str(zone_cible):
##ON ISOLE LE CHAMP CC
                            #print "CC DETECTED, i= ",i
                            #print zone_cible
                            count_Cc+=1
                            CCfield= re.search('<li><strong>Cc</strong>:(.*)<li><strong>Subject</strong>',str(zone_cible),flags=re.DOTALL)
                            CC= CCfield.group(1)
                            CC=str(CC)
                            CC=re.sub(r'^.+\n','',str(CC),flags=0)
                            CCs=re.findall("<script type=.*\n.+\n.+\n.+\n.+/script>", CC)
                            #sep1 = "\" + \"@\" + "
                            #mail1= mail1.split(sep1, 1)[0]
                            #print "CC FIELd\n"#,CCs
                            liste_CCs=[]
                            for CC in CCs:
                                #print CC,"\n___________________\n"
                                mail1= re.sub(r'<script type=\"text/javascript\">\n <!-- \n document\.write\(\"', '',str(CC))
                                mail2=re.sub(r'<script type=\"text/javascript\">\n <!-- \n document\.write\(\".+@\" \+ \"', '',str(CC))
                                sep1 = "\" + \"@\" + "
                                sep2 = "\")"
                                mail1= mail1.split(sep1, 1)[0].lower()
                                mail2= mail2.split(sep2, 1)[0].lower()
                                mail_CC= mail1+"@"+mail2
                                #print mail_CC,"\n___________\n"
                                if mail_CC not in liste_CCs :
                                    liste_CCs.append(mail_CC.lower())
                                if  mail_CC not in list_all_mails:
                                    list_all_mails.append(mail_CC.lower())
                                
                            #!print "CCs: ",liste_CCs
                            champs_mail["liste_CCs"] = liste_CCs
##ON ISOLE LE CHAMP DATE                       
                        #print "ZONECIBLE",zone_cible
                        champs_datef=re.search('<li><strong>Date</strong>:(.+?)</li>',str(zone_cible),flags=re.DOTALL)
                        #print "CHAMPSDATEF",champs_datef
                        try :
                            champs_date=champs_datef.group(1)
                        except :
                            #print "ZCparent", zone_cible.parent
                            champs_date=re.search('<!--X-Date: (.*) -->',str(soup))
                            champs_date=champs_date.group(1)
                            #print champs_date
                            
                        #print champs_date
                        try : 
                            champs_date=parser.parse(champs_date)
                        except ValueError :
                            sep=" ("
                            champs_date=champs_date.split(sep, 1)[0]
                            
                        #!print  champs_date
                        champs_mail["date"] = champs_date
                        
                        
                        dict_mails[os.path.join(subdir, file)]= champs_mail
                        break
##LE DICTIONNAIRE DES MAILS SE NOMME dict_mails
    #print dict_mails
    print count_message," messages"
    print count_Cc, " messages avec Cc"
    ratiocount = float(count_Cc)/float(count_message)
    print "Taux_de_mails_avec_CC:",ratiocount,"\n"
    return [dict_mails,list_all_mails]

In [108]:
def read_corpus_and_extract_topo_network(corpus):
    return

In [109]:
def read_corpus_and_extract_topo_Geo_network(corpus):
    return

In [110]:
def read_corpus_and_extract_mails(corpus):
    liste_mails=[]
    for mesg in corpus:
        #print corpus[mesg]
        if corpus[mesg]['mail_auteur'].lower() not in liste_mails:
            liste_mails.append(corpus[mesg]['mail_auteur'].lower())
        for i,key in enumerate(corpus[mesg]['liste_mails_dests']):
            #print key
            if key.lower() not in liste_mails:
                liste_mails.append(key.lower())
        if 'liste_CCs' in corpus[mesg] :
            #print "TRUVE CC"
            for i,key in enumerate(corpus[mesg]['liste_CCs']):
                #print key
                if key.lower() not in liste_mails:
                    liste_mails.append(key.lower())
    return liste_mails

In [111]:
def read_corpus_and_extract_localisations(corpus):
    return

In [112]:
def read_html_pages_in_dirs_and_testtextextract(pages_dir):
    dict_mails={}
    #rootdir="./DHFR_sample/"

    
    for subdir, dirs, files in os.walk(pages_dir):
#ON VA LIRE TOUS LES FICHIERS DU REP ROOTDIR
        
        for file in files:
            dictT={}
    #V1     if file.endswith('.html') == True  and file != ("index.html") and not re.match("mail\d+.html",file) : 
            if re.match("msg\d+.html",file):
                filename=os.path.join(subdir, file)
                with open(filename, 'r') as myfile:           
                
                    data=myfile.read()
                    result = re.search('<!--X-Body-of-Message-->(.*)<!--X-Body-of-Message-End-->', data, flags=re.DOTALL)
                    ZC= result.group(1)
                    ZC=cleanhtml(ZC)
                    dictT["content"]=ZC.lower()
                    dictT["geo"]=topic_analysis_with_manual_detection(ZC)
                dict_mails[filename]= dictT
            
    return dict_mails

In [113]:
def read_html_and_testtextextract(filename):
                dictT={}
                with open(filename, 'r') as myfile:
                    data=myfile.read()
                    result = re.search('<!--X-Body-of-Message-->(.*)<!--X-Body-of-Message-End-->', data, flags=re.DOTALL)
                    ZC= result.group(1)
                    ZC=cleanhtml(ZC)
                    dictT["content"]=ZC.lower()
                    dictT["geo"]=topic_analysis_with_manual_detection(ZC)
                return dictT

In [128]:
def read_corpus_and_extract_names(corpus):
    liste_noms=[]
    duplicate_list_noms=[]
    for mesg in corpus:
        try: 
            if corpus[mesg]['nom_auteur'][0]== "\"":
                print corpus[mesg]['nom_auteur']
                
        except IndexError:
            print "Bug", corpus[mesg]['nom_auteur']
            print corpus[mesg]
            
        if [corpus[mesg]['nom_auteur'].lower(),corpus[mesg]['mail_auteur'].lower()] not in liste_noms:
            for i,key in enumerate(liste_noms):
                if SequenceMatcher(None, corpus[mesg]['nom_auteur'], key[0]).ratio() >=0.9:
                    print "POSSIBLE DUPLICATE |", corpus[mesg]['nom_auteur'],"|   |",key[0],"|"
                    print corpus[mesg]['mail_auteur'], key[1]
                    if [[corpus[mesg]['nom_auteur'].lower(),corpus[mesg]['mail_auteur'].lower()],[key[0],key[1]]] not in duplicate_list_noms:
                        [x.lower() for x in corpus[mesg]['nom_auteur']]
                        [y.lower() for y in corpus[mesg]['mail_auteur']]
                        duplicate_list_noms.append([[corpus[mesg]['nom_auteur'],corpus[mesg]['mail_auteur']],[key[0],key[1]]])
                    
                
            liste_noms.append([corpus[mesg]['nom_auteur'],corpus[mesg]['mail_auteur']])
    
    return [liste_noms,duplicate_list_noms]

In [129]:
corpus_mails_DH=read_html_pages_in_dirs_and_extract_mails_dict("./DHFRsample")
#print corpus_mails_DH
liste_de_tous_les_mails=corpus_mails_DH[1]
corpus_mails_DH=corpus_mails_DH[0]



liste_mails=read_corpus_and_extract_mails(corpus_mails_DH)
print len(liste_mails)
print liste_mails


aude.da-cruz-lima
noiret, serge
noiret, serge
wandl-vogt, eveline
wandl-vogt, eveline
elisabeth.belmas
viera rebolledo-dhuin
ghislain sillaume
noiret, serge


KeyboardInterrupt: 

In [116]:
print len(liste_de_tous_les_mails)
print liste_de_tous_les_mails

278
['mehdi.khamassi@upmc.fr', 'dh@cru.fr', 'isabelle.thiebau@univ-lille2.fr', 'dh@groupes.renater.fr', 'colette.cadiou@irstea.fr', 'claire.clivaz@sib.swiss', 'johann.holland@campus-condorcet.fr', 'christine.michel@insa-lyon.fr', 'cerisier@univ-poitiers.fr', 'marjorie.burghart@ehess.fr', 'tei-fr@cru.fr', 'stephane.loret@univ-nantes.fr', 'quanti@groupes.renater.fr', 'rbdd@services.cnrs.fr', 'stephane.pouyllau@huma-num.fr', 'formation.continue@enc-sorbonne.fr', 'elenagonzalezblanco@yahoo.es', 'humanist@lists.digitalhumanities.org', 'globaloutlookdh-l@uleth.ca', 'digitalclassicist@jiscmail.ac.uk', 'dm-l@uleth.ca', 'dhd@mailman.rrz.uni-hamburg.de', 'aiucd-l@humnet.unipi.it', 'air-l@aoir.org', 'institute@lists.uvic.ca', 'tei-l@listserv.brown.edu', 'humanist-l@uleth.ca', 'centernet@lists.digitalhumanities.org', 'dhcarolina@listserv.unc.edu', 'humanidadesdigitais@gmail.com', 'website@hastac.org', 'southasia-dh@lists.globaloutlookdh.org', 'air-l@listserv.aoir.org', 'humanisticadh@gmail.com', '

In [117]:
import pickle
import sys
sys.setrecursionlimit(5000000)

with open('liste_de_tous_les_mails.obj', 'wb') as handle:
    pickle.dump(liste_de_tous_les_mails, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
with open('liste_de_tous_les_mails.obj', 'rb') as handle:
    liste_de_tous_les_mails = pickle.load(handle)

In [118]:
import pickle
import sys
sys.setrecursionlimit(5000000)

with open('corpus_mails_DH.obj', 'wb') as handle:
    pickle.dump(corpus_mails_DH, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [88]:
import pickle
with open('corpus_mails_DH.obj', 'rb') as handle:
    corpus_mails_DH = pickle.load(handle)

In [119]:
corpus_txt=read_html_pages_in_dirs_and_testtextextract("./DHFRsample")

In [123]:
print #corpus_txt
#lng= language_detection_with_pyenchant(str(corpus_txt))
#print lng

In [130]:
liste_noms_and_dups=read_corpus_and_extract_names(corpus_mails_DH)
print len(liste_noms_and_dups[0]),len(liste_noms_and_dups[1])
print liste_noms_and_dups[0]
print "________________________\n"
print liste_noms_and_dups[1]

POSSIBLE DUPLICATE | pierre mounier |   | pierre mounier |
pierre.mounier@openedition.org pierre.mounier@ehess.fr
POSSIBLE DUPLICATE | clarisse bardiot |   | clarisse bardiot |
clarisse_bardiot@me.com clarisse_bardiot@mac.com
POSSIBLE DUPLICATE | claire clivaz |   | claire clivaz |
claire.clivaz@sib.swiss claire.clivaz@unil.ch
POSSIBLE DUPLICATE | frédéric clavert |   | frédéric clavert |
frederic.clavert@uni.lu frederic@clavert.net
POSSIBLE DUPLICATE | stéphane pouyllau |   | stéphane pouyllau |
stephane.pouyllau@cnrs.fr stephane.pouyllau@huma-num.fr
POSSIBLE DUPLICATE | marin dacos |   | marin dacos |
marin.dacos@openedition.org marin.dacos@revues.org
POSSIBLE DUPLICATE | antonio casilli |   | antonio a. casilli |
antonio.casilli@googlemail.com antonio.casilli@ehess.fr
POSSIBLE DUPLICATE | sandrine clérisse |   | sandrine clérisse |
sandrine.clerisse@cnrs.fr sclerisse@parisnanterre.fr
POSSIBLE DUPLICATE | frédéric clavert |   | frédéric clavert |
frederic.clavert@unil.ch frederic@cla

In [145]:
def read_corpus_and_extract_names_2(corpus,liste_noms_and_dups):
    liste_noms=liste_noms_and_dups[0]
    dups=liste_noms_and_dups[1]
    dict_noms={}
    tmp_dups_dict={}
    tmp_dups_list=[]
    for dup in dups:
        dupA=dup[0]
        dupAnom=dup[0][0]
        dupAmail=dup[0][1]
        dupAdatemin=None
        dupAdatemax=None
        for mesg in corpus:
            #TEST MIN DATE
            [x.lower() for x in corpus[mesg]['mail_auteur']]
            if corpus[mesg]['mail_auteur']==dupAmail:
                if dupAdatemin is not None:
                    if corpus[mesg]['date']<dupAdatemin:
                        #print corpus[mesg]['date']
                        dupAdatemin=corpus[mesg]['date']
                else: dupAdatemin=corpus[mesg]['date']
             #TEST MAX DATE       
            if corpus[mesg]['mail_auteur']==dupAmail:
                if dupAdatemax is not None:
                    if corpus[mesg]['date']>dupAdatemax:
                        #print corpus[mesg]['date']
                        dupAdatemax=corpus[mesg]['date']
                else: dupAdatemax=corpus[mesg]['date']   
        
        
        dupB=dup[1]
        dupBnom=dup[1][0]
        dupBmail=dup[1][1]
        dupBdatemin=None
        dupBdatemax=None
        
        #TO TEST LATER WITH LIST_OF_NAMES_IN_ORDER_NOT_TO_ADD_DUPS_MULTIPLE_TIMES
        if dupAnom not in tmp_dups_list:
            tmp_dups_list.append(dupAnom)
        if dupBnom not in tmp_dups_list:
            tmp_dups_list.append(dupBnom)
        for mesg in corpus:
            #TEST MIN DATE
            if corpus[mesg]['mail_auteur']==dupBmail:
                if dupBdatemin is not None:
                    if corpus[mesg]['date']<dupBdatemin:
                        #print corpus[mesg]['date']
                        dupBdatemin=corpus[mesg]['date']
                else: dupBdatemin=corpus[mesg]['date']
             #TEST MAX DATE       
            if corpus[mesg]['mail_auteur']==dupBmail:
                if dupBdatemax is not None:
                    if corpus[mesg]['date']>dupBdatemax:
                        #print corpus[mesg]['date']
                        dupBdatemax=corpus[mesg]['date']
                else: dupBdatemax=corpus[mesg]['date'] 
        #print dupAmail," ",dupAdatemin,"/",dupAdatemax,"\n",dupBmail," ",dupBdatemin,"/",dupBdatemax,"\n"
        if bool(tmp_dups_dict)== False:
            tmp_dups_dict[dupAnom]=[[dupAmail,dupAdatemin,dupAdatemax],[dupBmail,dupBdatemin,dupBdatemax]]

            init=1
            
        if dupAnom not in tmp_dups_dict and init !=1:
            #CHECK PRINT
            #print dupAnom,"NOT IN\n",tmp_dups_dict
            
            tmp_dups_dict_tmp= dict.copy(tmp_dups_dict)
            for key in tmp_dups_dict_tmp:
                #print key
                
                if SequenceMatcher(None, dupAnom, key).ratio() <0.9:
                    tmp_dups_dict[dupAnom]=[[dupAmail,dupAdatemin,dupAdatemax],[dupBmail,dupBdatemin,dupBdatemax]]
                else: 
                    #print "HEERE"
                    tmp_dups_dict[key].append([[dupAmail,dupAdatemin,dupAdatemax],[dupBmail,dupBdatemin,dupBdatemax]])
        else:
            
            if init !=1:
                #print "HERE"
                tmp_dups_dict[dupAnom].append([[dupAmail,dupAdatemin,dupAdatemax],[dupBmail,dupBdatemin,dupBdatemax]])
        init=0
    
    #print     tmp_dups_dict
    tmp_nom_dict={}
    
    for nom in liste_noms:
        
        if nom[0] not in tmp_dups_list:
            if nom[0] in tmp_nom_dict:
                print "BUG NOMS"
                break
            else:
                nomdatemin=None
                nomdatemax=None
                for mesg in corpus:
            #TEST MIN DATE
                    if corpus[mesg]['mail_auteur']==nom[1]:
                        if nomdatemin is not None:
                            if corpus[mesg]['date']<nomdatemin:
                                #print corpus[mesg]['date']
                                nomdatemin=corpus[mesg]['date']
                        else: nomdatemin=corpus[mesg]['date']
                     #TEST MAX DATE       
                    if corpus[mesg]['mail_auteur']==nom[1]:
                        if nomdatemax is not None:
                            if corpus[mesg]['date']>nomdatemax:
                                #print corpus[mesg]['date']
                                nomdatemax=corpus[mesg]['date']
                        else: nomdatemax=corpus[mesg]['date']
                
                tmp_nom_dict[nom[0]]=[[nom[1],nomdatemin,nomdatemax]]
                #print nom[0]," ",nom[1]," ",nomdatemin,"/",nomdatemax,"\n"
                if bool(tmp_nom_dict)== False:
                    tmp_nom_dict[nom[0]]=[[nom[1],nomdatemin,nomdatemax]]

    #print  tmp_dups_dict,"\n_________________\n",tmp_nom_dict,'\n____________________________\n'    
    dict_noms_parses_dedoub_avec_date= tmp_dups_dict.copy()
    dict_noms_parses_dedoub_avec_date.update(tmp_nom_dict)
    print dict_noms_parses_dedoub_avec_date
    return dict_noms_parses_dedoub_avec_date
            
            
            
            
    
        
    
dict_noms_mails_debut_fin=read_corpus_and_extract_names_2(corpus_mails_DH,liste_noms_and_dups)   
print len(dict_noms_mails_debut_fin)

{'annael le poullennec': [['annael.le-poullennec@psl.eu', datetime.datetime(2018, 3, 1, 14, 3, 55, tzinfo=tzutc()), datetime.datetime(2018, 3, 1, 14, 3, 55, tzinfo=tzutc())]], 'marie-eglantine lescasse': [['marie.e.lescasse@gmail.com', datetime.datetime(2018, 3, 28, 17, 9, 57, tzinfo=tzoffset(None, 7200)), datetime.datetime(2018, 3, 28, 17, 9, 57, tzinfo=tzoffset(None, 7200))]], 'brunet mich\xc3\xa8le': [['iglouvremb@gmail.com', datetime.datetime(2014, 7, 17, 14, 17, 4, tzinfo=tzoffset(None, 7200)), datetime.datetime(2014, 7, 17, 14, 17, 4, tzinfo=tzoffset(None, 7200))]], 'irihs': [['irihs@univ-rouen.fr', datetime.datetime(2018, 3, 15, 9, 12, 21, tzinfo=tzoffset(None, 3600)), datetime.datetime(2018, 3, 15, 9, 12, 21, tzinfo=tzoffset(None, 3600))]], 'antonio casilli': [['antonio.casilli@googlemail.com', datetime.datetime(2014, 7, 7, 11, 23, 8, tzinfo=tzoffset(None, 7200)), datetime.datetime(2014, 7, 7, 11, 23, 8, tzinfo=tzoffset(None, 7200))], ['antonio.casilli@ehess.fr', datetime.datet

In [67]:
import pickle



with open('dict_noms_mails_debut_fin.obj', 'wb') as handle:
    pickle.dump(dict_noms_mails_debut_fin, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [44]:
import pickle
with open('dict_noms_mails_debut_fin.obj', 'rb') as handle:
    dict_noms_mails_debut_fin = pickle.load(handle)

In [ ]:
def extract_locations_from_email_adresses(liste_mails):
    import whois
    import time
    
 ###ON EXTRAIT LES DOMAINES_MAILS_ET_ON_VA_RECHERCHER_UNE_LOCALISATION_A_PARTIR_DE_CELA
###CE PROGRAMME EST EN DEUX PARTIES CAR ON DOIT D ABORD EXTRAIT LA LISTE DE WHOIS QUI LIMIT LE NOMBRE DE RESULTATS,
###ON ITERE DONC SUR UNE BOUCLE DE REMPLISSAGE D'UN DICT QU'ON RENVOIE, QU'ON PARSERA ENSUITE
    liste_domaines=[]
    #print liste_mails
    for key in  liste_mails:
        print key
        #print dict_noms[key]
        print str(key.split("@")[1:][0]).lower()
        if str(key.split("@")[1:][0].lower()) not in liste_domaines:
                    liste_domaines.append(key.split("@")[1:][0].lower())
            
    print liste_domaines
    dict_infos_domaines={}
    for domaine in liste_domaines:
        attempt=0
        not_localised=[]
        while True:
            try :
                if domaine not in dict_infos_domaines:
                    
                    infos_domaine = whois.whois(domaine)
                    print "\n______________\n",domaine,"\n"#,infos_domaine
                    dict_infos_domaines[domaine]=infos_domaine
                    time.sleep(4)
                    
                
                


            except Exception as e:
                attempt+=1
                time.sleep(15)
                print "ONE BREAK"
                #print e
                if attempt < 5:
                    continue
                else:
                    not_localised.append(domaine)
                    break
            break
    print dict_infos_domaines
    print not_localised
    return [dict_infos_domaines,not_localised]
            

    
        
list_localisations=extract_locations_from_email_adresses(liste_de_tous_les_mails)        

mehdi.khamassi@upmc.fr
upmc.fr
dh@cru.fr
cru.fr
isabelle.thiebau@univ-lille2.fr
univ-lille2.fr
dh@groupes.renater.fr
groupes.renater.fr
colette.cadiou@irstea.fr
irstea.fr
claire.clivaz@sib.swiss
sib.swiss
johann.holland@campus-condorcet.fr
campus-condorcet.fr
christine.michel@insa-lyon.fr
insa-lyon.fr
cerisier@univ-poitiers.fr
univ-poitiers.fr
marjorie.burghart@ehess.fr
ehess.fr
tei-fr@cru.fr
cru.fr
stephane.loret@univ-nantes.fr
univ-nantes.fr
quanti@groupes.renater.fr
groupes.renater.fr
rbdd@services.cnrs.fr
services.cnrs.fr
stephane.pouyllau@huma-num.fr
huma-num.fr
formation.continue@enc-sorbonne.fr
enc-sorbonne.fr
elenagonzalezblanco@yahoo.es
yahoo.es
humanist@lists.digitalhumanities.org
lists.digitalhumanities.org
globaloutlookdh-l@uleth.ca
uleth.ca
digitalclassicist@jiscmail.ac.uk
jiscmail.ac.uk
dm-l@uleth.ca
uleth.ca
dhd@mailman.rrz.uni-hamburg.de
mailman.rrz.uni-hamburg.de
aiucd-l@humnet.unipi.it
humnet.unipi.it
air-l@aoir.org
aoir.org
institute@lists.uvic.ca
lists.uvic.ca
tei-l

In [42]:
###ON SAUVEGARDE LES DONNNES EXTRAITES
import pickle

filehandler = open(b"list_localisations.obj","wb")
pickle.dump(list_localisations,filehandler)


In [8]:
###ON LES RECUPERE
import pickle


file = open("list_localisations.obj",'rb')
list_localisations = pickle.load(file)

IOError: [Errno 2] No such file or directory: 'list_localisations.obj'

In [89]:
###LES INFOS RECUPEREES DEPUIS WHOIS SONT MALFORMEES,ON NE PEUT LES CONVERTIR EN DICT PROPRE, IL FAUT DONC LES PARSER AVEC UN PARSEUR "MAISON"
#print list_localisations[0]


def extract_locations_from_email_adresses_2(dict_loc):
    list_locs=[]
   
    for loc in dict_loc:
        print loc
        #print str(dict_loc[loc])
        address_field=None
        city_field=None
        state_field=None
        country_field=None
        loc_full_string=""
        
        
        
        if "address" in  str(dict_loc[loc]):
            
            address_field= re.sub(r'.*\"address\": ','',str(dict_loc[loc]),flags=re.DOTALL)
            sep = ", \n  \""
            address_field = address_field.split(sep, 1)[0]
            #print address_field,"\n"
        #else:
            #print "NO ADDRESS FIELD\n"
        
        if "city" in  str(dict_loc[loc]):
            
            city_field= re.sub(r'.*\"city\": ','',str(dict_loc[loc]),flags=re.DOTALL)
            sep = ", \n  \""
            city_field = city_field.split(sep, 1)[0]
            #print city_field,"\n"
        #else:
            #print "NO city FIELD\n"
            
        if "state" in  str(dict_loc[loc]):
            
            state_field= re.sub(r'.*\"state\": ','',str(dict_loc[loc]),flags=re.DOTALL)
            sep = ", \n  \""
            state_field = state_field.split(sep, 1)[0]
            #print state_field,"\n"
        #else:
            #print "NO state FIELD\n"    
            
        
            
        if "country" in  str(dict_loc[loc]):
            
            country_field= re.sub(r'.*\"country\": ','',str(dict_loc[loc]),flags=re.DOTALL)
            sep = ", \n  \""
            country_field = country_field.split(sep, 1)[0]
            #print country_field,"\n"
        #else:
            #print "NO country FIELD\n"
            
        # on concatene le tout, qu'on passera à geopy pour identifier des addresses
        loc_full_string=loc
        if address_field is not None :
            loc_full_string+=" "+address_field
        if  city_field is not None:
            loc_full_string+=" "+city_field
            
        if  state_field is not None:
            loc_full_string+=" "+state_field
        if  country_field is not None:
            
            loc_full_string+=" "+country_field
        print loc_full_string
        list_locs.append([loc,loc_full_string])
    
    return list_locs
        
        
        
         
        
        
        
        
        
            
            
            
        
    
    
    
    
    

    
    
list_loc_with_whois=extract_locations_from_email_adresses_2(list_localisations[0])    

univ-orleans.fr
univ-orleans.fr
design.ulaval.ca
design.ulaval.ca
inha.fr
inha.fr
me.com
me.com "1 Infinite Loop" "Cupertino" "CA" "US"
dariah.eu
dariah.eu
cnam.fr
cnam.fr
lecnam.net
lecnam.net "Rue des Cray\ufffdres, BP 1034" "reims" null "FR"
univ-nantes.fr
univ-nantes.fr
mae.u-paris10.fr
mae.u-paris10.fr
irit.fr
irit.fr
univ-brest.fr
univ-brest.fr
univ-lorraine.fr
univ-lorraine.fr
icp.fr
icp.fr
oeaw.ac.at
oeaw.ac.at "Dr. Ignaz Seipel-Platz 2" "Wien" "Austria"
u-paris10.fr
u-paris10.fr
inria.fr
inria.fr
enc-sorbonne.fr
enc-sorbonne.fr
univ-amu.fr
univ-amu.fr
sciencespo-lyon.fr
sciencespo-lyon.fr
univ-tours.fr
univ-tours.fr
univ-rouen.fr
univ-rouen.fr
univ-reims.fr
univ-reims.fr
campus-condorcet.fr
campus-condorcet.fr
clavert.net
clavert.net "Obfuscated whois Gandi-63-65 boulevard Massena" "Obfuscated whois Gandi-Paris" "Paris" "FR"
ehess.fr
ehess.fr
univ-paris1.fr
univ-paris1.fr
unistra.fr
unistra.fr
yahoo.es
yahoo.es null null null null
univ-lyon3.fr
univ-lyon3.fr
revues.org
revues.

In [ ]:
def retrieve_geocodes_from_list_loc(lists_locs_aff):
#FOR OSM API
    from geopy.geocoders import Nominatim
    import time
    import os
    import os.path
    import pickle
    
    
#FOR GOOGLE API 
    from geopy.geocoders import GoogleV3
    
    my_api_key = os.environ.get('MY_API_KEY', None)
    
    print my_api_key
    
    
    loc_dict={}
    #need to export export MY_API_KEY="" from terminal
    google_maps = GoogleV3(api_key=my_api_key,domain='maps.google.fr')
    for loc_af in lists_locs_aff:
        
        loc_aff=loc_af[1].replace('\"'," ").replace("[","").replace("]","").replace(" null ", " ").replace(","," ").replace("\\ufffd","è")
        loc_name=loc_af[0]
        
        if os.path.exists("./geocod_tmp_obj/"+loc_name+".obj")==True:
            with open("./geocod_tmp_obj/"+loc_name+".obj", 'rb') as handle:
                location = pickle.load(handle)
                loc_dict[loc_name]=location
                
        else:        
            print loc_aff
                #geolocator = Nominatim()
                #location = geolocator.geocode(loc_aff)

            location=google_maps.geocode(loc_aff,"exactly_one=True")
            try:
                        print location.raw
                        #time.sleep(1)
                        loc_dict[loc_name]=location.raw
                        

                        filehandler = open("./geocod_tmp_obj/"+loc_name+".obj","wb")
                        pickle.dump(location.raw,filehandler)

            except :
                        loc_dict[loc_name]="TO8BE8MANUALLY8EDITED"
        
            
  
        
        
    
                
             
        
    
    return loc_dict
            
            
            
        
    

geocoded_locs=retrieve_geocodes_from_list_loc(list_loc_with_whois)

In [118]:
import pickle



with open('geocoded_locs.obj', 'wb') as handle:
    pickle.dump(geocoded_locs, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [44]:
import pickle
with open('geocoded_locs.obj', 'rb') as handle:
    geocoded_locs = pickle.load(handle)

In [ ]:
print geocoded_locs


In [ ]:
##RETRIEVE THE  LOCATION DATAS THAT NEED TO BE MANUALLY EDITED
#"TO8BE8MANUALLY8EDITED"
def extract_fields_requiring_manual_intervention(geocoded_locs):
    fields_needing_manual_geoloc=[]
    for loc in geocoded_locs:

        print loc
        print geocoded_locs[loc]
        if geocoded_locs[loc]=="TO8BE8MANUALLY8EDITED":
            fields_needing_manual_geoloc.append(loc)
    return fields_needing_manual_geoloc


location_needing_manual_intervention=extract_fields_requiring_manual_intervention(geocoded_locs)

In [27]:
##AND EXPORT IT TO CSV
print location_needing_manual_intervention
import csv

with open("./CSVs_for_manual_fixing/location_refining.csv", 'wb') as myfile:
    
    rows = zip(location_needing_manual_intervention)
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    for row in rows:
            wr.writerow(row)

['lecnam.net', 'ahicf.com', 'clavert.net', 'yahoo.es', 'infoclio.ch', 'adbu.fr', 'histographe.com', 'openedition.org', 'aalto.fi', 'gmail.com', 'helsinki.fi', 'uni.lu', 'revues.org']


In [68]:
##AFTER CORRECTION,READ IT BACK TO COMPlETE PARSING (1stcol=name,second col=lat,long)
##PRODUCE LAT/LNG FIELDS AS FOLLOW
##'mines.org': {u'geometry': {u'location': {u'lat': 48.845585, u'lng': 2.339632}}}
#FOR "NOTRELEVANT" fields,so far we will use geocord 45.3896523,-5.4866776,8.25, in the atlantic ocean (gmail accouns and such)
#WE will randomly spread it to distinguish from anon accounts.
def correct_geolocs_by_hand():
    import csv
    import random
    dict_loc_dicts_to_replace_TO8BE8MANUALLY8EDITED_in_geocoded_locs={}
    with open("./CSVs_for_manual_fixing/location_refining.csv", 'rb') as myfile:
    
    
        wr = csv.reader(myfile)
        #print wr
        for row in wr:
                latlngdict= {u'geometry': {u'location': {u'lat': None, u'lng': None}}}
         #       print row[0],"/","("+row[1]+")"
                latrow=re.sub(r'', '',str(row[1]))
                sep = ","
                latrow= latrow.split(sep, 1)[0]
          #      print latrow
                
                if latrow == "NOTRELEVANT" :
                    #ADD RANDON 0.03 TO NOTRELEVLATLNG
                    latlngdict[u'geometry'][u'location'][u'lat']= 45.3896523+random.random()
                    latlngdict[u'geometry'][u'location'][u'lng']= 2.339632+random.random()
                else:
                    
                    lngrow=re.sub(r'.*,', '',str(row[1]))
                    #sep = ","
                    #latrow= latrow.split(sep, 1)[0]
           #         print lngrow
                    latlngdict[u'geometry'][u'location'][u'lat']=latrow
                    latlngdict[u'geometry'][u'location'][u'lng']=lngrow
                    
                
                
                
            #    print latlngdict
                dict_loc_dicts_to_replace_TO8BE8MANUALLY8EDITED_in_geocoded_locs[unicode(row[0])]= latlngdict
                

    return dict_loc_dicts_to_replace_TO8BE8MANUALLY8EDITED_in_geocoded_locs


dict_de_localisations_de_remplacement= correct_geolocs_by_hand()
print dict_de_localisations_de_remplacement



{u'lecnam.net': {u'geometry': {u'location': {u'lat': '48.8667495', u'lng': '2.3532426'}}}, u'histographe.com': {u'geometry': {u'location': {u'lat': '48.8667495', u'lng': '2.3532426'}}}, u'revues.org': {u'geometry': {u'location': {u'lat': '43.3430737', u'lng': '5.4328468'}}}, u'adbu.fr': {u'geometry': {u'location': {u'lat': '48.8440177', u'lng': '2.3374428'}}}, u'clavert.net': {u'geometry': {u'location': {u'lat': '46.5210932', u'lng': '6.5779719'}}}, u'openedition.org': {u'geometry': {u'location': {u'lat': '43.3430737', u'lng': '5.4328468'}}}, u'yahoo.es': {u'geometry': {u'location': {u'lat': 45.766883438059835, u'lng': 2.5993781799139635}}}, u'helsinki.fi': {u'geometry': {u'location': {u'lat': '60.1779949', u'lng': '24.8543427'}}}, u'aalto.fi': {u'geometry': {u'location': {u'lat': '60.1866719', u'lng': '24.8254933'}}}, u'gmail.com': {u'geometry': {u'location': {u'lat': 45.79559208545012, u'lng': 2.7429090515033137}}}, u'uni.lu': {u'geometry': {u'location': {u'lat': '49.5042575', u'lng'

In [ ]:
def dict_correction_by_dict(dict_to_correct,value_to_find,dict_with_corrections_arranged_by_keys):
    dict_corrected=dict.copy(dict_to_correct)
    keys = []
    
    for key in dict_with_corrections_arranged_by_keys:
        
            #print key
            keys.append(unicode(key))
    print keys, "___??_________"
    keystash = keys
    for dict_name in dict_to_correct:
        
        print dict_name,"!!!???"
        #print dict_to_correct[unicode(dict_name)]
        
        
        #print unicode(dict_to_correct[unicode(dict_name)]),"----!!----"
        if unicode(dict_name) in keys :
                keystash.remove(unicode(dict_name))
            #if dict_to_correct[unicode(dict_name)] == value_to_find:
                print "FOUND IT!!!!!!!!!!!!!!!!!!!!!!!"
                dict_corrected[unicode(dict_name)]= dict_with_corrections_arranged_by_keys[key]
                print "DICT CORRIGE",dict_corrected[unicode(dict_name)]
            #else:
            #    print "BIGBUG as match"
            #    break
    ###KEYSTASH IS TO ADD MANUALLY REPLACEMENT ADRESSES    
    print keystash
    if len(keystash) >0:
        for keyst in keystash:
            dict_corrected[keyst]= dict_with_corrections_arranged_by_keys[keyst]
    print dict_corrected
    return dict_corrected
    

dict_corrected_DH= dict_correction_by_dict(geocoded_locs,"TO8BE8MANUALLY8EDITED",dict_de_localisations_de_remplacement)

In [ ]:
print dict_corrected_DH

In [87]:
import pickle



with open('dict_corrected_DH.obj', 'wb') as handle:
    pickle.dump(dict_corrected_DH, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [56]:
import pickle
with open('dict_corrected_DH.obj', 'rb') as handle:
    dict_corrected_DH = pickle.load(handle)

In [ ]:
print dict_corrected_DH 

In [2]:
def stats_corpus():
    

SyntaxError: unexpected EOF while parsing (<ipython-input-2-e725142f834e>, line 1)

In [28]:
import random
print random.random()

0.0351086459122


In [49]:
import enchant
print enchant.list_languages()

SyntaxError: invalid syntax (<ipython-input-49-29a123259797>, line 41)

In [ ]:
Analyse =topic_analysis_with_nltk_gensim(corpus_txt)

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
print(Analyse)

In [ ]:
def topic_analysis_with_nltk_gensim(string_to_read):
    string_to_read=unicode(str(string_to_read), 'utf8')
    lng = language_detection_with_pyenchant(string_to_read)
   
    from nltk.corpus import stopwords
    from nltk.stem.wordnet import WordNetLemmatizer
    import string
    print lng
    stop = set(stopwords.words(lng))
    exclude = set(string.punctuation)
    lemma = WordNetLemmatizer()
    def clean(doc):
        stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
        
        punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
        normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
        return normalized

    doc_clean = clean(string_to_read).split()
    print "DOC CLEAN",doc_clean
        # Importing Gensim
    import gensim
    from gensim import corpora

        # Creating the term dictionary of our courpus, where every unique term is assigned an index. 
    dictionary = corpora.Dictionary([doc_clean])

        # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
    doc_term_matrix = dictionary.doc2bow(doc_clean)
    Lda = gensim.models.ldamodel.LdaModel
    print Lda

        # Running and Trainign LDA model on the document term matrix.
    ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=5)
    return ldamodel
       
        
    
    
    
    #return [topic_words,doctopic]